In [2]:
#! /usr/bin/python

In [ ]:
#previous line makes this an executable python file. Execution line = ./UNIPROT_data_mining_simplified_cluster.py

In [13]:
from bs4 import BeautifulSoup
from urllib.request import Request, urlopen
import re as re
import io

In [4]:
import ssl

ssl._create_default_https_context = ssl._create_unverified_context

In [16]:
#import data
import numpy as np 
import pandas as pd
import csv


In [17]:
#Load the Uniprot (genefamilies) or Metacyc (pathways) dataframe produced from HumanN2 

path_data = pd.read_csv("D:/all_genefamilies_unstratified.csv")
print("Uniprot IDs found in metagenomes loaded")
path_data.head(5)

Uniprot IDs found in metagenomes loaded


,ID,ZD-10_R1.fastq.paired_genefamilies,ZD-11_R1.fastq.paired_genefamilies,ZD-1_R1.fastq.paired_genefamilies,ZD-2_R1.fastq.paired_genefamilies,ZD-3_R1.fastq.paired_genefamilies,ZD-5_R1.fastq.paired_genefamilies,ZD-6_R1.fastq.paired_genefamilies,ZD-7_R1.fastq.paired_genefamilies,ZD-9_R1.fastq.paired_genefamilies,ZR-10_R1.fastq.paired_Abundance-RPKs,ZR-11_R1.fastq.paired_Abundance-RPKs,ZR-1_R1.fastq.paired_Abundance-RPKs,ZR-2_R1.fastq.paired_Abundance-RPKs,ZR-3_R1.fastq.paired_Abundance-RPKs,ZR-5_R1.fastq.paired_Abundance-RPKs,ZR-6_R1.fastq.paired_Abundance-RPKs,ZR-7_R1_PE.paired_Abundance-RPKs,ZR-9_R1.fastq.paired_Abundance-RPKs
0,UNMAPPED,1.084452e+07,1.371072e+07,7.816504e+06,1.545008e+07,2.027130e+07,7.132913e+06,8.967164e+06,2.681140e+07,8.689543e+06,17130715.0,14079971.0,1.557884e+07,14044010.0,12026731.0,13345739.0,16452394.0,11096073.0,17724912.0
1,UniRef50_A0A008BVE2,0.000000e+00,0.000000e+00,6.032935e-01,0.000000e+00,1.446397e+00,0.000000e+00,0.000000e+00,3.125681e-01,0.000000e+00,0.0,0.0,0.000000e+00,0.0,0.0,0.0,0.0,0.0,0.0
2,UniRef50_A0A008BXX0: ThiW protein,6.550574e+00,0.000000e+00,6.020868e+00,0.000000e+00,9.623164e+00,4.123694e+00,8.213207e+00,1.081012e+01,8.509572e+00,0.0,0.0,9.487334e-01,0.0,0.0,0.0,0.0,0.0,0.0
3,UniRef50_A0A008CGV2,1.223591e+00,1.001678e+01,2.887737e+00,0.000000e+00,0.000000e+00,0.000000e+00,5.490426e+00,7.844199e-01,0.000000e+00,0.0,0.0,0.000000e+00,0.0,0.0,0.0,0.0,0.0,0.0
4,UniRef50_A0A008CL44: Adenosylmethionine-8-amin...,0.000000e+00,0.000000e+00,0.000000e+00,3.035417e+00,1.022402e+00,0.000000e+00,0.000000e+00,2.159804e+00,0.000000e+00,0.0,0.0,0.000000e+00,0.0,0.0,0.0,0.0,0.0,0.0


In [7]:
#subset of only five lines of the orginal DF for code testing before running entire dataframe
path_data = path_data.head(50)

#Clean up the ID for generating the URL in a new column
path_data['URL'] = path_data['ID'].str.split(':').str[0].str.replace('UniRef50_','')

path_data.head(5)


,ID,ZD-10_R1.fastq.paired_genefamilies,ZD-11_R1.fastq.paired_genefamilies,ZD-1_R1.fastq.paired_genefamilies,ZD-2_R1.fastq.paired_genefamilies,ZD-3_R1.fastq.paired_genefamilies,ZD-5_R1.fastq.paired_genefamilies,ZD-6_R1.fastq.paired_genefamilies,ZD-7_R1.fastq.paired_genefamilies,ZD-9_R1.fastq.paired_genefamilies,ZR-10_R1.fastq.paired_Abundance-RPKs,ZR-11_R1.fastq.paired_Abundance-RPKs,ZR-1_R1.fastq.paired_Abundance-RPKs,ZR-2_R1.fastq.paired_Abundance-RPKs,ZR-3_R1.fastq.paired_Abundance-RPKs,ZR-5_R1.fastq.paired_Abundance-RPKs,ZR-6_R1.fastq.paired_Abundance-RPKs,ZR-7_R1_PE.paired_Abundance-RPKs,ZR-9_R1.fastq.paired_Abundance-RPKs,URL
0,UNMAPPED,1.084452e+07,1.371072e+07,7.816504e+06,1.545008e+07,2.027130e+07,7.132913e+06,8.967164e+06,2.681140e+07,8.689543e+06,17130715.0,14079971.0,1.557884e+07,14044010.0,12026731.0,13345739.0,16452394.0,11096073.0,17724912.0,UNMAPPED
1,UniRef50_A0A008BVE2,0.000000e+00,0.000000e+00,6.032935e-01,0.000000e+00,1.446397e+00,0.000000e+00,0.000000e+00,3.125681e-01,0.000000e+00,0.0,0.0,0.000000e+00,0.0,0.0,0.0,0.0,0.0,0.0,A0A008BVE2
2,UniRef50_A0A008BXX0: ThiW protein,6.550574e+00,0.000000e+00,6.020868e+00,0.000000e+00,9.623164e+00,4.123694e+00,8.213207e+00,1.081012e+01,8.509572e+00,0.0,0.0,9.487334e-01,0.0,0.0,0.0,0.0,0.0,0.0,A0A008BXX0
3,UniRef50_A0A008CGV2,1.223591e+00,1.001678e+01,2.887737e+00,0.000000e+00,0.000000e+00,0.000000e+00,5.490426e+00,7.844199e-01,0.000000e+00,0.0,0.0,0.000000e+00,0.0,0.0,0.0,0.0,0.0,0.0,A0A008CGV2
4,UniRef50_A0A008CL44: Adenosylmethionine-8-amin...,0.000000e+00,0.000000e+00,0.000000e+00,3.035417e+00,1.022402e+00,0.000000e+00,0.000000e+00,2.159804e+00,0.000000e+00,0.0,0.0,0.000000e+00,0.0,0.0,0.0,0.0,0.0,0.0,A0A008CL44


In [8]:
#if the file doesn't have a column for the URL already you can use this to make a new column in the dataframe

#for the metacyc pathways 
#path_data['URL.link'] = 'https://biocyc.org/META/NEW-IMAGE?object=' + path_data['ID'].astype(str) 
#path_data.head(10)

#for the uniprot 
path_data['URL.link'] = 'https://www.uniprot.org/uniprot/' + path_data['URL'].astype(str) + '.txt?version=3'
path_data['URL.link'].head(10)

0    https://www.uniprot.org/uniprot/UNMAPPED.txt?v...
1    https://www.uniprot.org/uniprot/A0A008BVE2.txt...
2    https://www.uniprot.org/uniprot/A0A008BXX0.txt...
3    https://www.uniprot.org/uniprot/A0A008CGV2.txt...
4    https://www.uniprot.org/uniprot/A0A008CL44.txt...
5    https://www.uniprot.org/uniprot/A0A008HDX8.txt...
6    https://www.uniprot.org/uniprot/A0A008HNV8.txt...
7    https://www.uniprot.org/uniprot/A0A008HZX3.txt...
8    https://www.uniprot.org/uniprot/A0A008J5V2.txt...
9    https://www.uniprot.org/uniprot/A0A008JDR7.txt...
Name: URL.link, dtype: object

In [9]:
# remove Unmapped and unintigrated rows
path_data = path_data[~path_data.ID.str.contains("UNMAPPED")]
path_data = path_data[~path_data.ID.str.contains("UNITEGRATED")]

path_data.head(5)

,ID,ZD-10_R1.fastq.paired_genefamilies,ZD-11_R1.fastq.paired_genefamilies,ZD-1_R1.fastq.paired_genefamilies,ZD-2_R1.fastq.paired_genefamilies,ZD-3_R1.fastq.paired_genefamilies,ZD-5_R1.fastq.paired_genefamilies,ZD-6_R1.fastq.paired_genefamilies,ZD-7_R1.fastq.paired_genefamilies,ZD-9_R1.fastq.paired_genefamilies,...,ZR-11_R1.fastq.paired_Abundance-RPKs,ZR-1_R1.fastq.paired_Abundance-RPKs,ZR-2_R1.fastq.paired_Abundance-RPKs,ZR-3_R1.fastq.paired_Abundance-RPKs,ZR-5_R1.fastq.paired_Abundance-RPKs,ZR-6_R1.fastq.paired_Abundance-RPKs,ZR-7_R1_PE.paired_Abundance-RPKs,ZR-9_R1.fastq.paired_Abundance-RPKs,URL,URL.link
1,UniRef50_A0A008BVE2,0.000000,0.00000,0.603294,0.000000,1.446397,0.000000,0.000000,0.312568,0.000000,...,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,A0A008BVE2,https://www.uniprot.org/uniprot/A0A008BVE2.txt...
2,UniRef50_A0A008BXX0: ThiW protein,6.550574,0.00000,6.020868,0.000000,9.623164,4.123694,8.213207,10.810124,8.509572,...,0.0,0.948733,0.0,0.0,0.0,0.0,0.0,0.0,A0A008BXX0,https://www.uniprot.org/uniprot/A0A008BXX0.txt...
3,UniRef50_A0A008CGV2,1.223591,10.01678,2.887737,0.000000,0.000000,0.000000,5.490426,0.784420,0.000000,...,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,A0A008CGV2,https://www.uniprot.org/uniprot/A0A008CGV2.txt...
4,UniRef50_A0A008CL44: Adenosylmethionine-8-amin...,0.000000,0.00000,0.000000,3.035417,1.022402,0.000000,0.000000,2.159804,0.000000,...,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,A0A008CL44,https://www.uniprot.org/uniprot/A0A008CL44.txt...
5,UniRef50_A0A008HDX8: Membrane protein,0.000000,0.00000,0.000000,0.000000,0.000000,0.000000,0.000000,2.891548,0.000000,...,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,A0A008HDX8,https://www.uniprot.org/uniprot/A0A008HDX8.txt...


In [17]:
#test the URL status to avoid 404 http error
import requests 
from multiprocessing.dummy import Pool as ThreadPool 

urls = path_data['URL.link']

def url_access(x):
    for url in path_data['URL.link']:
        response= requests.get(url)
        status= response.status_code
        path_data['Status']=status

        return requests.head(x).status_code



#use this to run prior code in parallel with multiple threads    
if __name__ == "__main__":
    pool = ThreadPool(4)  # Make the Pool of workers (specific number of threads you want to use)
    results = pool.map(url_access, urls) #Open the urls in their own threads
    pool.close() #close the pool and wait for the work to finish 
    pool.join() 
    

In [18]:
path_data['Status'].value_counts()
#we are looking for nothing in the 400 range

200    49
Name: Status, dtype: int64

In [19]:
#The heavy lifting code that extracts the taxonomic data. 
import requests
import time
from multiprocessing.dummy import Pool as ThreadPool 

startTime = time.time()


#change to different name of the column that has the URLs
urls = path_data['URL.link']#.head(15)
new_df = []

#this def will run a the following commands when the command get_status is called
def get_status(url):
    try:
        req = Request(url)
        html_page = urlopen(req)
        soup = BeautifulSoup(html_page, "html.parser")
        html_text = soup.get_text()
        f = open("html_text.txt", "w",encoding='utf-8')
        for line in html_text:
            f.write(line)
        f.close()

        #html_text = html_text.replace("\n", "") # this was needed for the metacyc not the Uniprot
        #return html_text #This if not uncommented out will return the html page, this is a control

        #pattern = re.findall('{}(.*?){}'.format('Range:', 'Synonyms:'), html_text) # metacyc
        pattern = re.findall('{}(.*?){}'.format('OC\s+', '\n'), html_text) #uniprot, Do not replace teh \n with nothing
        #print(pattern)
        #print(len(pattern))
        if len(pattern)==0:# alternate search for pathway data. These if statements find the taxa from metacyc.org
            pattern = re.findall('{}(.*?){}'.format('Range:', 'BioCyc ID:'), html_text)
        #    print(pattern)
            if len(pattern)==0:
                pattern = re.findall('{}(.*?){}'.format('Range:', 'Note'),html_text)  
        new_df.append(pattern) 
        return pattern # this will return the string of interest. 
    except:
        return'Error Invalid URL bypassing, overriding error'
        new_df.append(x)
        pass
if __name__ == "__main__":
    pool = ThreadPool(4)  # Make the Pool of workers (specific number of threads in this case 2)
    results = pool.map(get_status, urls) #Open the urls in their own threads
    pool.close() #close the pool and wait for the work to finish 
    pool.join() 
    
print('Completed multithreaded taxonomic data mininng:\n')
#print(results) # this is the result of the html collection
#print(new_df)
print(len(new_df))

executionTime = (time.time() - startTime)
print('\nExecution time in seconds: ' + str(executionTime)) 


Execution time in seconds: 27.663142681121826


In [20]:
#Verify output
print(results)
print(len(new_df))
path_data.head(15)

[['Bacteria; Firmicutes; Bacilli; Bacillales; Staphylococcus.'], ['Bacteria; Firmicutes; Bacilli; Bacillales; Staphylococcus.'], ['Bacteria; Firmicutes; Bacilli; Bacillales; Staphylococcus.'], ['Bacteria; Firmicutes; Bacilli; Bacillales; Staphylococcus.'], ['Bacteria; Firmicutes; Bacilli; Bacillales; Staphylococcus.'], ['Bacteria; Firmicutes; Bacilli; Bacillales; Staphylococcus.'], ['Bacteria; Firmicutes; Bacilli; Bacillales; Staphylococcus.'], ['Bacteria; Firmicutes; Bacilli; Bacillales; Staphylococcus.'], ['Bacteria; Firmicutes; Bacilli; Bacillales; Staphylococcus.'], ['Bacteria; Firmicutes; Bacilli; Bacillales; Staphylococcus.'], ['Bacteria; Firmicutes; Bacilli; Bacillales; Staphylococcus.'], ['Bacteria; Firmicutes; Bacilli; Bacillales; Staphylococcus.'], ['Bacteria; Firmicutes; Bacilli; Bacillales; Staphylococcus.'], ['Bacteria; Firmicutes; Bacilli; Bacillales; Staphylococcus.'], ['Bacteria; Firmicutes; Bacilli; Bacillales; Staphylococcus.'], ['Bacteria; Firmicutes; Bacilli; Bacill

,ID,ZD-10_R1.fastq.paired_genefamilies,ZD-11_R1.fastq.paired_genefamilies,ZD-1_R1.fastq.paired_genefamilies,ZD-2_R1.fastq.paired_genefamilies,ZD-3_R1.fastq.paired_genefamilies,ZD-5_R1.fastq.paired_genefamilies,ZD-6_R1.fastq.paired_genefamilies,ZD-7_R1.fastq.paired_genefamilies,ZD-9_R1.fastq.paired_genefamilies,...,ZR-1_R1.fastq.paired_Abundance-RPKs,ZR-2_R1.fastq.paired_Abundance-RPKs,ZR-3_R1.fastq.paired_Abundance-RPKs,ZR-5_R1.fastq.paired_Abundance-RPKs,ZR-6_R1.fastq.paired_Abundance-RPKs,ZR-7_R1_PE.paired_Abundance-RPKs,ZR-9_R1.fastq.paired_Abundance-RPKs,URL,URL.link,Status
1,UniRef50_A0A008BVE2,0.000000,0.000000,0.603294,0.000000,1.446397,0.000000,0.000000,0.312568,0.000000,...,0.000000,0.0,0.000000,0.0,0.0,0.0,0.000000,A0A008BVE2,https://www.uniprot.org/uniprot/A0A008BVE2.txt...,200
2,UniRef50_A0A008BXX0: ThiW protein,6.550574,0.000000,6.020868,0.000000,9.623164,4.123694,8.213207,10.810124,8.509572,...,0.948733,0.0,0.000000,0.0,0.0,0.0,0.000000,A0A008BXX0,https://www.uniprot.org/uniprot/A0A008BXX0.txt...,200
3,UniRef50_A0A008CGV2,1.223591,10.016780,2.887737,0.000000,0.000000,0.000000,5.490426,0.784420,0.000000,...,0.000000,0.0,0.000000,0.0,0.0,0.0,0.000000,A0A008CGV2,https://www.uniprot.org/uniprot/A0A008CGV2.txt...,200
4,UniRef50_A0A008CL44: Adenosylmethionine-8-amin...,0.000000,0.000000,0.000000,3.035417,1.022402,0.000000,0.000000,2.159804,0.000000,...,0.000000,0.0,0.000000,0.0,0.0,0.0,0.000000,A0A008CL44,https://www.uniprot.org/uniprot/A0A008CL44.txt...,200
5,UniRef50_A0A008HDX8: Membrane protein,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,2.891548,0.000000,...,0.000000,0.0,0.000000,0.0,0.0,0.0,0.000000,A0A008HDX8,https://www.uniprot.org/uniprot/A0A008HDX8.txt...,200
6,UniRef50_A0A008HNV8,0.000000,0.000000,0.000000,0.000000,2.380601,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.0,0.000000,0.0,0.0,0.0,0.000000,A0A008HNV8,https://www.uniprot.org/uniprot/A0A008HNV8.txt...,200
7,UniRef50_A0A008HZX3,18.130932,0.000000,2.041180,0.000000,24.476229,5.335423,14.416726,14.293369,9.066296,...,0.598185,0.0,0.615486,0.0,0.0,0.0,2.573751,A0A008HZX3,https://www.uniprot.org/uniprot/A0A008HZX3.txt...,200
8,UniRef50_A0A008J5V2,2.450980,1.027523,0.690127,0.000000,3.526915,4.031627,1.916746,2.766148,2.550371,...,0.000000,0.0,0.000000,0.0,0.0,0.0,0.000000,A0A008J5V2,https://www.uniprot.org/uniprot/A0A008J5V2.txt...,200
9,UniRef50_A0A008JDR7,0.000000,0.593379,0.684974,0.000000,2.577922,0.778610,0.000000,2.883224,0.000000,...,0.000000,0.0,0.000000,0.0,0.0,0.0,0.000000,A0A008JDR7,https://www.uniprot.org/uniprot/A0A008JDR7.txt...,200
10,UniRef50_A0A008JKD8,1.383134,0.000000,1.071622,0.000000,6.774332,0.000000,0.000000,15.118708,5.376344,...,0.000000,0.0,0.000000,0.0,0.0,0.0,0.000000,A0A008JKD8,https://www.uniprot.org/uniprot/A0A008JKD8.txt...,200


In [27]:
#pandas function: adds the new_df of the taxa into the existing df while creating new column
df = path_data.assign(taxonomy=new_df)
#df

In [30]:
#This makes a new column that converts the list/array data generated previouly into a str which i can then reogranize
df['taxonomy'] = df['taxonomy'].astype(str)
df

,ID,ZD-10_R1.fastq.paired_genefamilies,ZD-11_R1.fastq.paired_genefamilies,ZD-1_R1.fastq.paired_genefamilies,ZD-2_R1.fastq.paired_genefamilies,ZD-3_R1.fastq.paired_genefamilies,ZD-5_R1.fastq.paired_genefamilies,ZD-6_R1.fastq.paired_genefamilies,ZD-7_R1.fastq.paired_genefamilies,ZD-9_R1.fastq.paired_genefamilies,...,ZR-2_R1.fastq.paired_Abundance-RPKs,ZR-3_R1.fastq.paired_Abundance-RPKs,ZR-5_R1.fastq.paired_Abundance-RPKs,ZR-6_R1.fastq.paired_Abundance-RPKs,ZR-7_R1_PE.paired_Abundance-RPKs,ZR-9_R1.fastq.paired_Abundance-RPKs,URL,URL.link,Status,taxonomy
1,UniRef50_A0A008BVE2,0.000000,0.000000,0.603294,0.000000,1.446397,0.000000,0.000000,0.312568,0.000000,...,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,A0A008BVE2,https://www.uniprot.org/uniprot/A0A008BVE2.txt...,200,['Bacteria; Firmicutes; Bacilli; Bacillales; S...
2,UniRef50_A0A008BXX0: ThiW protein,6.550574,0.000000,6.020868,0.000000,9.623164,4.123694,8.213207,10.810124,8.509572,...,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,A0A008BXX0,https://www.uniprot.org/uniprot/A0A008BXX0.txt...,200,['Bacteria; Firmicutes; Bacilli; Bacillales; S...
3,UniRef50_A0A008CGV2,1.223591,10.016780,2.887737,0.000000,0.000000,0.000000,5.490426,0.784420,0.000000,...,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,A0A008CGV2,https://www.uniprot.org/uniprot/A0A008CGV2.txt...,200,['Bacteria; Firmicutes; Bacilli; Bacillales; S...
4,UniRef50_A0A008CL44: Adenosylmethionine-8-amin...,0.000000,0.000000,0.000000,3.035417,1.022402,0.000000,0.000000,2.159804,0.000000,...,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,A0A008CL44,https://www.uniprot.org/uniprot/A0A008CL44.txt...,200,['Bacteria; Firmicutes; Bacilli; Bacillales; S...
5,UniRef50_A0A008HDX8: Membrane protein,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,2.891548,0.000000,...,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,A0A008HDX8,https://www.uniprot.org/uniprot/A0A008HDX8.txt...,200,['Bacteria; Firmicutes; Bacilli; Bacillales; S...
6,UniRef50_A0A008HNV8,0.000000,0.000000,0.000000,0.000000,2.380601,0.000000,0.000000,0.000000,0.000000,...,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,A0A008HNV8,https://www.uniprot.org/uniprot/A0A008HNV8.txt...,200,['Bacteria; Firmicutes; Bacilli; Bacillales; S...
7,UniRef50_A0A008HZX3,18.130932,0.000000,2.041180,0.000000,24.476229,5.335423,14.416726,14.293369,9.066296,...,0.0,0.615486,0.000000,0.000000,0.000000,2.573751,A0A008HZX3,https://www.uniprot.org/uniprot/A0A008HZX3.txt...,200,['Bacteria; Firmicutes; Bacilli; Bacillales; S...
8,UniRef50_A0A008J5V2,2.450980,1.027523,0.690127,0.000000,3.526915,4.031627,1.916746,2.766148,2.550371,...,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,A0A008J5V2,https://www.uniprot.org/uniprot/A0A008J5V2.txt...,200,['Bacteria; Firmicutes; Bacilli; Bacillales; S...
9,UniRef50_A0A008JDR7,0.000000,0.593379,0.684974,0.000000,2.577922,0.778610,0.000000,2.883224,0.000000,...,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,A0A008JDR7,https://www.uniprot.org/uniprot/A0A008JDR7.txt...,200,['Bacteria; Firmicutes; Bacilli; Bacillales; S...
10,UniRef50_A0A008JKD8,1.383134,0.000000,1.071622,0.000000,6.774332,0.000000,0.000000,15.118708,5.376344,...,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,A0A008JKD8,https://www.uniprot.org/uniprot/A0A008JKD8.txt...,200,['Bacteria; Firmicutes; Bacilli; Bacillales; S...


In [31]:
m = df['taxonomy'].str.contains('Bacteria|Archaea')

#df1 now contains everything but Bacteria and Archaea
df1 = df[~m].reset_index(drop=True) 
df1

,ID,ZD-10_R1.fastq.paired_genefamilies,ZD-11_R1.fastq.paired_genefamilies,ZD-1_R1.fastq.paired_genefamilies,ZD-2_R1.fastq.paired_genefamilies,ZD-3_R1.fastq.paired_genefamilies,ZD-5_R1.fastq.paired_genefamilies,ZD-6_R1.fastq.paired_genefamilies,ZD-7_R1.fastq.paired_genefamilies,ZD-9_R1.fastq.paired_genefamilies,...,ZR-2_R1.fastq.paired_Abundance-RPKs,ZR-3_R1.fastq.paired_Abundance-RPKs,ZR-5_R1.fastq.paired_Abundance-RPKs,ZR-6_R1.fastq.paired_Abundance-RPKs,ZR-7_R1_PE.paired_Abundance-RPKs,ZR-9_R1.fastq.paired_Abundance-RPKs,URL,URL.link,Status,taxonomy


In [32]:
#this is will produce the df that has Bacteria and Archaea
df2 = df[m].reset_index(drop=True)
df2

,ID,ZD-10_R1.fastq.paired_genefamilies,ZD-11_R1.fastq.paired_genefamilies,ZD-1_R1.fastq.paired_genefamilies,ZD-2_R1.fastq.paired_genefamilies,ZD-3_R1.fastq.paired_genefamilies,ZD-5_R1.fastq.paired_genefamilies,ZD-6_R1.fastq.paired_genefamilies,ZD-7_R1.fastq.paired_genefamilies,ZD-9_R1.fastq.paired_genefamilies,...,ZR-2_R1.fastq.paired_Abundance-RPKs,ZR-3_R1.fastq.paired_Abundance-RPKs,ZR-5_R1.fastq.paired_Abundance-RPKs,ZR-6_R1.fastq.paired_Abundance-RPKs,ZR-7_R1_PE.paired_Abundance-RPKs,ZR-9_R1.fastq.paired_Abundance-RPKs,URL,URL.link,Status,taxonomy
0,UniRef50_A0A008BVE2,0.000000,0.000000,0.603294,0.000000,1.446397,0.000000,0.000000,0.312568,0.000000,...,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,A0A008BVE2,https://www.uniprot.org/uniprot/A0A008BVE2.txt...,200,['Bacteria; Firmicutes; Bacilli; Bacillales; S...
1,UniRef50_A0A008BXX0: ThiW protein,6.550574,0.000000,6.020868,0.000000,9.623164,4.123694,8.213207,10.810124,8.509572,...,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,A0A008BXX0,https://www.uniprot.org/uniprot/A0A008BXX0.txt...,200,['Bacteria; Firmicutes; Bacilli; Bacillales; S...
2,UniRef50_A0A008CGV2,1.223591,10.016780,2.887737,0.000000,0.000000,0.000000,5.490426,0.784420,0.000000,...,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,A0A008CGV2,https://www.uniprot.org/uniprot/A0A008CGV2.txt...,200,['Bacteria; Firmicutes; Bacilli; Bacillales; S...
3,UniRef50_A0A008CL44: Adenosylmethionine-8-amin...,0.000000,0.000000,0.000000,3.035417,1.022402,0.000000,0.000000,2.159804,0.000000,...,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,A0A008CL44,https://www.uniprot.org/uniprot/A0A008CL44.txt...,200,['Bacteria; Firmicutes; Bacilli; Bacillales; S...
4,UniRef50_A0A008HDX8: Membrane protein,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,2.891548,0.000000,...,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,A0A008HDX8,https://www.uniprot.org/uniprot/A0A008HDX8.txt...,200,['Bacteria; Firmicutes; Bacilli; Bacillales; S...
5,UniRef50_A0A008HNV8,0.000000,0.000000,0.000000,0.000000,2.380601,0.000000,0.000000,0.000000,0.000000,...,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,A0A008HNV8,https://www.uniprot.org/uniprot/A0A008HNV8.txt...,200,['Bacteria; Firmicutes; Bacilli; Bacillales; S...
6,UniRef50_A0A008HZX3,18.130932,0.000000,2.041180,0.000000,24.476229,5.335423,14.416726,14.293369,9.066296,...,0.0,0.615486,0.000000,0.000000,0.000000,2.573751,A0A008HZX3,https://www.uniprot.org/uniprot/A0A008HZX3.txt...,200,['Bacteria; Firmicutes; Bacilli; Bacillales; S...
7,UniRef50_A0A008J5V2,2.450980,1.027523,0.690127,0.000000,3.526915,4.031627,1.916746,2.766148,2.550371,...,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,A0A008J5V2,https://www.uniprot.org/uniprot/A0A008J5V2.txt...,200,['Bacteria; Firmicutes; Bacilli; Bacillales; S...
8,UniRef50_A0A008JDR7,0.000000,0.593379,0.684974,0.000000,2.577922,0.778610,0.000000,2.883224,0.000000,...,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,A0A008JDR7,https://www.uniprot.org/uniprot/A0A008JDR7.txt...,200,['Bacteria; Firmicutes; Bacilli; Bacillales; S...
9,UniRef50_A0A008JKD8,1.383134,0.000000,1.071622,0.000000,6.774332,0.000000,0.000000,15.118708,5.376344,...,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,A0A008JKD8,https://www.uniprot.org/uniprot/A0A008JKD8.txt...,200,['Bacteria; Firmicutes; Bacilli; Bacillales; S...


In [24]:
df['taxonomy.str'].value_counts()

'bacteria, proteobacteria, gammaproteobacteria, pseudomonadales,', 'moraxellaceae, acinetobacter,', 'acinetobacter calcoaceticus/baumannii complex.'    49
Name: taxonomy.str, dtype: int64

In [190]:
#save the dataframe to a location with all the taxonomy data
#You can partition the taxa data first either in python or Notepad++ to convert the ";" --> "," for csv 
#you would then have to make new labels for each taxanomic rank
df1.to_csv("/home/FCAM/jgil/Beth_files/non_bacterial_mallard_path_abund_taxa.csv")
df2.to_csv("/home/FCAM/jgil/Beth_files/bacterial_mallard_path_abund_taxa.csv")